In [90]:
import os, sys
sys.path.insert(0, '/home/jtorrenc/mvesc/ETL')
from mvesc_utility_functions import *
import numpy as np
import pandas as pd
import random
from functools import partial

In [116]:
def color_gt(val,x=.5,c='blue'):
    """
    Colors values greater than x blue (or other provided color)
    
    :param float x: lower bound on colored values
    :param str c: color
    :returns: css string to color values in dataframe
    :rtype: str
    """
    color = c if val > x else 'black'
    return 'color: %s' % color

def table_exists(cursor, table, schema='clean'):
    """
    Checks to see if a table exists in the database
    
    :param pg.cursor cursor: 
    :param str table:
    :param str schema:
    :rtype: bool
    """
    cursor.execute("""                                                   
            select count(*) from information_schema.tables                       
            where table_schema = %s and table_name = %s
            """, [schema, table])
    return cursor.fetchall()[0][0]

In [129]:
with postgres_pgconnection_generator() as connection:
    with connection.cursor() as cursor:
        cursor.execute('select distinct model_name from model.reports')
        model_list = [m[0] for m in cursor.fetchall()]
        best = dict()
        for m in model_list:
            cursor.execute("""select filename from model.reports where model_name = %s
                            order by val_precision_10 desc limit 1; """, [m])
            best[m] = cursor.fetchall()[0][0]
    predictions = pd.DataFrame()
    for m, f in best.items():
        df = read_table_to_df(connection, f,columns = ['student_lookup','predicted_score'],
                                          schema='predictions')
        df.set_index('student_lookup',inplace=True)
        df.columns = [m]
        predictions = pd.concat((predictions,df), axis=1, join='inner')


In [120]:
kendall = predictions.corr('kendall')
spearman = predictions.corr('spearman')

# Kendall's Tau Prediction Correlation

In [121]:
kendall.style.applymap(partial(color_gt,x=.65,c='blue'))

# Spearman's Prediction Correlation

In [122]:
spearman.style.applymap(partial(color_gt,x=.75,c='blue'))

In [174]:
with postgres_pgconnection_generator() as connection:
    with connection.cursor() as cursor:
        cursor.execute('select distinct model_name from model.reports')
        model_list = [m[0] for m in cursor.fetchall()]
        cursor.execute('select distinct feature_categories from model.reports')
        feature_list = [f[0] for f in cursor.fetchall()]
        feature_list = [f if ',' not in f  for f in feature_list]
        best = dict()
        features = dict()
        for f in feature_list:
            best[f] = dict()
            for m in model_list:
                cursor.execute("""select filename from model.reports 
                                where model_name = %s
                                    and feature_categories = %s
                                    and feature_grades like '5, 6, 7, 8, 9'
                                order by val_precision_10 desc limit 1; """, [m,f])
#                best[f][m] = 
                print(m, f)
                print(cursor.fetchall()[0][0])
            temp = pd.DataFrame()
            for m, file in best[f].items():
                if table_exists(cursor, file, 'feature_scores'):
                    df = read_table_to_df(connection, file ,columns=['feature','importance'],schema='feature_scores')
                    df.set_index('feature',inplace=True)
                    df.columns = [m]
                    temp = pd.concat((temp,df), axis=1, join='inner')
                    features[f] = temp

SyntaxError: invalid syntax (<ipython-input-174-6ce9a8c6aa7d>, line 7)

In [154]:
kendall_f = features['absence'].corr('kendall')
spearman_f = features['absence'].corr('spearman')   

# Kendall's Feature Correlation


In [155]:
kendall_f.style.applymap(partial(color_gt,x=.65,c='blue'))

# Spearman's Feature Correlation


In [157]:
spearman_f.style.applymap(partial(color_gt,x=.8,c='blue'))

In [158]:
features['absence']

,GB,AB,RF,DT,SVM,SGD,ET,logit
feature,,,,,,,,
absence_gr_9,0.318075,0.594929,0.575692,0.577405,0.000303,1.624373,0.356194,0.710943
tardy_gr_9,0.062354,0.100142,0.043160,0.023294,-0.000018,-2.188515,0.024161,0.108634
tardy_unexcused_gr_9,0.027932,0.035173,0.045942,0.019650,0.000016,-2.854402,0.023865,-0.052338
absence_consec_gr_9,0.066715,0.152964,0.097122,0.008563,0.000050,1.588377,0.047326,0.046690
absence_unexcused_gr_9,0.082752,0.024632,0.185050,0.053947,-0.000035,0.952575,0.115825,0.017850
